In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#DTL Data Transform and Load
from scipy.io import arff

#Sampling
from sklearn.model_selection import train_test_split

#Prediction algorthim
from sklearn.linear_model import LogisticRegression

#for confusion matrix to find TP TN FP FN
from sklearn import metrics

#for accuracy, precision and other evaluation metrics
from sklearn.metrics import classification_report

In [46]:
data, meta = arff.loadarff('/Users/ali/Downloads/phishing+websites/Training Dataset.arff')

In [47]:
df=pd.DataFrame(data)
str_df = df.select_dtypes([object])
str_df = str_df.stack().str.decode('utf-8').unstack()
df = str_df

In [48]:
df.to_excel('phising-uci-dataset.xlsx', header=True)

In [49]:
X= df[['having_IP_Address', 'URL_Length', 'Shortining_Service',
       'having_At_Symbol', 'double_slash_redirecting', 'Prefix_Suffix',
       'having_Sub_Domain', 'SSLfinal_State', 'Domain_registeration_length',
       'Favicon', 'port', 'HTTPS_token', 'Request_URL', 'URL_of_Anchor',
       'Links_in_tags', 'SFH', 'Submitting_to_email', 'Abnormal_URL',
       'Redirect', 'on_mouseover', 'RightClick', 'popUpWidnow', 'Iframe',
       'age_of_domain', 'DNSRecord', 'web_traffic', 'Page_Rank',
       'Google_Index', 'Links_pointing_to_page', 'Statistical_report']]

y=df['Result']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 42)

In [53]:
logreg=LogisticRegression(random_state=42)
logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)

In [55]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[ 864,   92],
       [  76, 1179]])

In [62]:
target_names = ['Not phishing', 'Phishing']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

Not phishing       0.92      0.90      0.91       956
    Phishing       0.93      0.94      0.93      1255

    accuracy                           0.92      2211
   macro avg       0.92      0.92      0.92      2211
weighted avg       0.92      0.92      0.92      2211



In [64]:
# Convert labels from {-1, 1} to {0, 1}
y_test_binary = (y_test + 1) // 2  # Convert -1 to 0 and 1 to 1

# Calculate ROC curve and AUC
fpr, tpr, _ = metrics.roc_curve(y_test_binary, y_pred_proba)
auc = metrics.roc_auc_score(y_test_binary, y_pred_proba)

# Plot ROC curve
# Convert AUC value to string before concatenating
plt.plot(fpr, tpr, label="data 1, auc=" + str(auc))

plt.legend(loc=4)
plt.show()


TypeError: can only concatenate str (not "int") to str